# Lazy evaluation and graphs

## Just in Time calculation or Pure Python

Doing a simple calculation and generating a value as it could be found in any function (like calculating the NLL)

In [1]:
import tensorflow as tf

tf.compat.v1.disable_eager_execution()

In [2]:
const_5 = 5
const_3 = 3
sum_5_3 = const_5 + const_3

const_7 = 7
const_2 = 2
sum_7_2 = const_7 + const_2

prod_sums = sum_5_3 * sum_7_2

We get an object out of the calculations:

In [3]:
prod_sums

72

Unfortunately, this was not efficient and we cannot deduce any more information form the output about it (like where it came from, equivalent to doing a simple physics exercise algebraicly versus directly entering the numbers into every variable)

## Lazy evaluation

So it would be better, to "build" the calculation in the first place. Therefore, we need so-called "lazy evaluation", an object that first gets composed and run afterwards.

Let's do this in python

Two equivalent ways of writing a function in python

In [4]:
def func():
    return 42
func = lambda: 42

In [5]:
const_5 = lambda: 5
const_3 = lambda: 3
sum_5_3 = lambda: const_5() + const_3()

const_7 = lambda: 7
const_2 = lambda: 2
sum_7_2 = lambda: const_7() + const_2()

prod_sums = lambda: sum_5_3() * sum_7_2()

And now we did not yet evaluate anything.

In [6]:
prod_sums

<function __main__.<lambda>()>

To evaluate this object, we simply call it:

In [7]:
prod_sums()

72

The advantage over the previous approach: we _could_ use the information stored in prod_sums to improve the calculation _before_ we run it.

In [8]:
import inspect
lines = inspect.getsource(prod_sums)
print(lines)

prod_sums = lambda: sum_5_3() * sum_7_2()



## Graphs

Of course, this was a simple example. Instead of setting up something like this, let's use an implementation: TensorFlow

In [9]:
from zfit import ztf

In [10]:
const_5 = ztf.constant(5)
const_3 = ztf.constant(3)
sum_5_3 = const_5 + const_3  # equivalent to the below one
# sum_5_3 = tf.add(const_5, const_3)

const_7 = ztf.constant(7)
const_2 = ztf.constant(2)
sum_7_2 = const_7 + const_2

prod_sums = sum_5_3 * sum_7_2

In [11]:
prod_sums

<tf.Tensor 'mul:0' shape=() dtype=float64>

Before we hat a function, that was our lazy evaluatable object, now it's a Tensor. Names don't matter here.

To run it, instead of putting parenthesis around, we use something else

In [12]:
import zfit

In [13]:
zfit.run(prod_sums)

72.0

And now let's look at the graph!

In [14]:
prod_sums  # output from the operation...

<tf.Tensor 'mul:0' shape=() dtype=float64>

In [15]:
prod_sums.op  # multiplies the inputs:

<tf.Operation 'mul' type=Mul>

In [16]:
prod_sums.op.inputs[:]  # with for example input 0 from the op..

[<tf.Tensor 'add:0' shape=() dtype=float64>,
 <tf.Tensor 'add_1:0' shape=() dtype=float64>]

In [17]:
prod_sums.op.inputs[0].op  # the first add with inputs

<tf.Operation 'add' type=AddV2>

In [18]:
prod_sums.op.inputs[0].op.inputs[:]

[<tf.Tensor 'Const_1:0' shape=() dtype=float64>,
 <tf.Tensor 'Const_2:0' shape=() dtype=float64>]

We have the whole definition at hand! It is a simple matter of implementations to, for example, check if a value depends on another (by recursively searching its inputs if it is there).

More useful: using the chain rule, we are able to compute the derivative of **any** value with respect to **any** other value:

In [19]:
derivate_sum_by_const5 = tf.gradients(prod_sums, const_5)

In [20]:
derivate_sum_by_const5

[<tf.Tensor 'gradients/mul_grad/Mul:0' shape=() dtype=float64>]

In [21]:
zfit.run(derivate_sum_by_const5)

[9.0]

What we didn't noticed: The whole code was also parallelized automatically on any available CPU/GPU.